### Subetapa 2.1.1: Lidar com dados nulos. Por exemplo, preencher valores nulos de notas com 0 ou a média.
### Subetapa 2.1.2: Normalizar as colunas de texto (nomes de cursos, etc.) para evitar inconsistências (por exemplo, converter para letras maiúsculas ou minúsculas).
### Subetapa 2.1.3: Validar e corrigir os tipos de dados. Garantir que a coluna de notas seja do tipo numérico (float ou int).

In [14]:
import pandas as pd

dataset = '../data/student_grades.csv'
df = pd.read_csv(dataset)

In [16]:
df.fillna(0) # Preenche todos os valores nulos com "0" 
# Apesar de já sabermos pela análise anterior que não temos valores nulos.

,Student ID,Course,Grade
0,1,Math,F
1,1,Science,C
2,1,History,A
3,1,English,F
4,1,Physical Education,C
...,...,...,...
49995,10000,Math,D
49996,10000,Science,F
49997,10000,History,B
49998,10000,English,F


In [18]:
df['Course'].unique() # Verifica quais são os cursos que temos

array(['Math', 'Science', 'History', 'English', 'Physical Education'],
      dtype=object)

In [20]:
dc_course = { # Cria um dicionario para traduzir as matérias
    'Math': 'matematica',
    'Science': 'ciencias',
    'History': 'historia',
    'English': 'ingles',
    'Physical Education': 'educacao fisica'
}


df['Course'].replace(dc_course) # Testa a implementação, mas não aplica
# df.head() retorna para o ingles, portanto precisamos salvar as modificações
df['Course'] = df['Course'].replace(dc_course) # Agora sim, salvamos as mudanças
# df.head() retorna os valores atualizados


In [22]:
# Mesmo procedimento com as notas agora
df['Grade'].unique() 

array(['F', 'C', 'A', 'B', 'D'], dtype=object)

In [24]:
dc_grade = { # Cria dicionário para notas, trocando para valores numéricos
    'A':10,
    'B':8,
    'C':6,
    'D':4,
    'E':2,
    'F':0
}

df['Grade'] = df['Grade'].replace(dc_grade) # Atualizando novos valores
df.head() 

,Student ID,Course,Grade
0,1,matematica,0
1,1,ciencias,6
2,1,historia,10
3,1,ingles,0
4,1,educacao fisica,6


In [26]:
# Agora vamos mudar o texto das colunas em si.
dc_col_names = {
    'Student ID': 'IdAluno',
    'Course': 'curso',
    'Grade': 'nota'
}

df = df.rename(columns= dc_col_names) # Renomeia as colunas

df.head()

,IdAluno,curso,nota
0,1,matematica,0
1,1,ciencias,6
2,1,historia,10
3,1,ingles,0
4,1,educacao fisica,6


In [28]:
df.info() # Percebi que a coluna de notas ainda estava no formato de object, precisamos corrigir isso.
# As notas devem ser número de ponto flutuante.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   IdAluno  50000 non-null  int64 
 1   curso    50000 non-null  object
 2   nota     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [30]:
df['nota'] = df['nota'].astype(float) # Convertendo o tipo da coluna para float
df.info() # Verifica a mudança de object para float
df.head() # verifica a mudança 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   IdAluno  50000 non-null  int64  
 1   curso    50000 non-null  object 
 2   nota     50000 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.1+ MB


,IdAluno,curso,nota
0,1,matematica,0.0
1,1,ciencias,6.0
2,1,historia,10.0
3,1,ingles,0.0
4,1,educacao fisica,6.0


### Subetapa 2.2.1: Criar uma conexão com o banco de dados SQLite dentro do seu Notebook.
### Subetapa 2.2.2: Definir o nome da tabela que irá receber os dados, por exemplo, tabela_notas.
### Subetapa 2.2.3: Utilizar o pandas para carregar o DataFrame já tratado para a tabela SQLite.
#### df_tratado.to_sql('tabela_notas', conn, if_exists='replace', index=False)
### Subetapa 2.2.4: Verificar se a tabela foi criada e populada corretamente, executando uma simples consulta SQL (SELECT * FROM tabela_notas LIMIT 5).

In [32]:
import sqlite3 as sql

In [34]:
conn = sql.connect('../db/dev.db') 
# Cria a conexão com o banco de dados sqlite na pasta db, gerando autmomaticamente o dev.db

In [36]:
df.to_sql('tabela_notas', conn, if_exists='replace', index=False) # Converte nosso DataFrame do arquivo csv em uma tabela no banco de dados SQLite

# 'tabela_notas': nome para a tabela.
# conn: variável que guarda a conexão com o banco de dados.
# if_exists='replace': diz ao pandas o que fazer se a tabela já existir, ele vai substituir a tabela.
# index=False: O pandas, por padrão, tenta salvar o índice do DataFrame como uma coluna no banco de dados. 
# o índice não tem nenhum significado especial, no nosso exemplo, então vou ignorá-lo.

50000

In [54]:
cursor = conn.cursor() # cursor é a ferramenta que permite executar comandos SQL

cursor.execute("SELECT * FROM tabela_notas LIMIT 5").fetchall() # Testando se a população foi feita corretamente

[(1, 'matematica', 0.0),
 (1, 'ciencias', 6.0),
 (1, 'historia', 10.0),
 (1, 'ingles', 0.0),
 (1, 'educacao fisica', 6.0)]

In [59]:
cursor.execute("SELECT * FROM tabela_notas LIMIT 5") # necessário rodar de novo sem o fetchall para retornar o print corretamente
# Porque?
# O fetchall funciona assim:
# Imagine que o cursor é uma caixa de correio.
# Quando você usa o .execute(), é como se você tivesse enviado uma carta (SELECT * FROM tabela_notas) para os correios (SQLite). A carta pede um pacote com as 5 primeiras notas.
# Os correios enviam o pacote para a sua caixa de correio (cursor). O pacote está lá, mas ainda está fechado.
# Quando você usa o .fetchall(), é como se você estivesse abrindo o pacote e tirando todas as correspondências de uma só vez.
# Portanto o fetchall esvazia a sua caixa de correios (cursor).
resultados = cursor.fetchall() 
print(resultados)

[(1, 'matematica', 0.0), (1, 'ciencias', 6.0), (1, 'historia', 10.0), (1, 'ingles', 0.0), (1, 'educacao fisica', 6.0)]
